# WIP Stub Notebook

This example has not been implemented yet. Track
[this github issue](https://github.com/jupyrdf/ipyradiant/issues/67) for more
information.

## Example Process (Custom)

For custom process, show replacing the `voc:starship` with the `ex:startshipName` via
`voc:starship/rdfs:label`.

### Nodes

We can overwwrite the `cls.sparql` attribute of each query class in order to change the
behavior of the `rdf2lpg` process.

> Note: Make sure to overwrite the `cls.initNs` if custom namespaces are used.

### Edges

Similar to [Nodes](#Nodes), we can overwwrite the `cls.sparql` attribute of each query
class in order to change the behavior of the `rdf2lpg` process.

> Note: Make sure to overwrite the `cls.initNs` if custom namespaces are used.